In [1]:
# pip install google-api-python-client
# !pip install isodate

In [2]:
# !pip install google-api-python-client

In [23]:
from googleapiclient.discovery import build
import pandas as pd
from tqdm.notebook import tqdm
import isodate
from datetime import datetime
from pandas_summary import DataFrameSummary
import json
import os
import glob

# Function the Fetches the data of the unique channels

In [24]:
import dateutil.relativedelta
from datetime import date
sats = []
numMonths = 6

def get_sub(channelId):
    return youtube.channels().list(id = channelId,  part = 'statistics').execute()['items'][0]['statistics']['subscriberCount']

def get_youtube_data(channel_name = None, channelId = None,  verbose = False):
    if channelId == None:
        assert channel_name != None, "You must have a channel Name or a Channel ID"
        snippets = youtube.search().list(part = "snippet", type = "channel", q = channel_name).execute()
        channelId =  snippets['items'][0]['snippet']['channelId']
        
    content = youtube.channels().list(id = channelId, part = "contentDetails").execute()
    uploadId = content['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    allVid = []
    nextPageToken = None
    page_no = 0
    
    print("Fetching Videos in Channel")
    while True:
        res = youtube.playlistItems().list(
            part = "snippet",maxResults= 50, playlistId = uploadId, pageToken = nextPageToken).execute()
        allVid += res['items']
        page_no += 1
        nextPageToken = res.get('nextPageToken')
        if verbose:
            print(f"Checking Video page {page_no}")
            
        latest_date = pd.to_datetime(allVid[-1]['snippet']['publishedAt']).tz_localize(None).date()
        three_months_b4 = date.today() - dateutil.relativedelta.relativedelta(months=numMonths)
        if ( latest_date < three_months_b4): # If content is more than 3 months old then stop scraping
            print(f"{numMonths} Months Exceeded")
            break
        if nextPageToken is None: # If there is no more content left to scrape then stop scraping
            break
        
    name, title,publishedDate, thumbnails, subcriber_count_list, view_count,desc,duration, like = [],[],[],[],[],[],[],[],[]
    subscriber_count = get_sub(channelId)

    print("Fetching Video MetaData")
    for i in tqdm(range(0, len(allVid))):
        title.append((allVid[i])['snippet']['title'])
        publishedDate.append((allVid[i])['snippet']['publishedAt'])
        thumbnails.append((allVid[i])['snippet']['thumbnails']['high']['url'])
        subcriber_count_list.append(subscriber_count) 
        video_id = (allVid[i])['snippet']['resourceId']['videoId']
        stats = youtube.videos().list(part = "statistics",maxResults= 1, id = video_id
                                      ).execute()['items'][0]['statistics']
        contentDeets = youtube.videos().list(part = "contentDetails",maxResults= 1, id = video_id
                                      ).execute()['items'][0]['contentDetails']
        view_count.append( stats['viewCount'])
        desc.append((allVid[i])['snippet']['description'])
        duration.append(str(isodate.parse_duration(contentDeets['duration'])))
        like.append(stats['likeCount'])
        name.append((allVid[i])['snippet']["channelTitle"])
    data = {"Channel Name":name, "title":title, "date":publishedDate, "thumbnailUrl": thumbnails, 
            "Views":view_count, "Description":desc, 
            "Duration":duration,"Like":like, "Subscribers":subcriber_count_list}
    df = pd.DataFrame(data)
    df["date"]= pd.to_datetime(df["date"])
    df["date"]= df["date"].dt.tz_localize(None)
    return df

# Add Your Api Key here

In [25]:
"""
To get the API Key follow these steps 
    1) go to google console 
    2) Create a new Project 
    3) Add youtube data api v3 and enable it 
    4) Manage —> credentials —> Create credentials 
    5) It will create an api that refreshes everyday at 4pm
    6) Qouta is 10k
"""
with open('secrets.json', 'r') as f:
    SECRETS = json.load(f)

youTubeApiKey = SECRETS["YOUTUBE_API_KEY"]

youtube = build('youtube','v3', developerKey=youTubeApiKey)

# Searches for the unique channels that return when you search bitcoin

In [26]:
top_bitcoin_channels = youtube.search().list(part = "snippet", q = "crypto", maxResults = 10).execute()
# top_bitcoin_channels['items']

In [27]:
potential_channel_dict = {}
for item in tqdm(top_bitcoin_channels['items']):
    potential_channel_dict[(item['snippet']['channelTitle'])] = item['snippet']['channelId']

  0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
potential_channel_dict.keys()

dict_keys(['CryptosRUs', 'BitBoy Crypto', 'Eric Krown Crypto', 'CRYPTO', 'Michael Pizzino', 'Crypto Banter', 'Greg Dickerson', 'CNBC Television'])

In [29]:
blackListChannels = ['Tesla [Live Elon Musk]','CNBC Television','Kitco NEWS','Naeem','Jason Pizzino','Cardano CEO','Crypto Fanta','ARK Invest ','Savvy Finance',
                    "BTC Moneymaker", "Nathalie Bitcoinsensus", "Cryptolyze", "Enter The Crypto Matrix","Wolves of Crypto", "Crypto Dog","Kuch New For You"]

In [30]:
for channel in blackListChannels:
    if channel in potential_channel_dict.keys():
        del potential_channel_dict[channel]

# Loops through all unique channels and gets data for each 

In [31]:
import os
from googleapiclient.errors import HttpError

potential_channel_Ids = list(potential_channel_dict.values())
potential_channel_Name = list(potential_channel_dict.keys())
verbose_exception = True

for i in range(len(potential_channel_Ids)):
    temp_df = None
    try:
        if not os.path.isfile(f"./final_data_folder/{potential_channel_Name[i]}.xlsx"): # Will only fetch if the file has not been pulled yet
            print(f'\nLooking at {potential_channel_Name[i]} YouTube Channel')
            temp_df = get_youtube_data(channel_name = potential_channel_Ids[i], verbose = False)
            temp_df.to_excel(f"./final_data_folder/{potential_channel_Name[i]}.xlsx")
        else:
            print(f'\n{potential_channel_Name[i]} File already exists')
    except HttpError as e:
        print("You probably ran out of your daily qouta \n")
        if verbose_exception:
            print(e)
        break
    except Exception as p:
        print('Some other error')
        if verbose_exception:
            print(p)



CryptosRUs File already exists

BitBoy Crypto File already exists

Eric Krown Crypto File already exists

CRYPTO File already exists

Michael Pizzino File already exists

Crypto Banter File already exists

Greg Dickerson File already exists


# Combines all the unique excels in to 1 

In [32]:
# getting excel files to be merged from the Desktop 
cwd = os.getcwd() 
path = os.path.join(cwd +"\\final_data_folder" )


# read all the files with extension .xlsx i.e. excel 
filenames = glob.glob(path + "\*.xlsx")
# print('File names:', filenames)

# empty data frame for the new output excel file with the merged excel files
outputxlsx = pd.DataFrame()

# for loop to iterate all excel files
for file in filenames:
    # using concat for excel files
    # after reading them with read_excel()
    df = pd.concat(pd.read_excel( file, sheet_name=None), ignore_index=True, sort=False)

    # appending data of excel files
    outputxlsx = outputxlsx.append( df, ignore_index=True)

if not os.path.isdir(path + "\\Combined"):
    os.mkdir(path + "\\Combined")
outputxlsx.to_excel(path + "\\Combined\\Output.xlsx", index=False)
print('Final Excel sheet now generated in the combined folder:')

Final Excel sheet now generated in the combined folder:


In [33]:
outputxlsx['Channel Name'].nunique()

114

In [34]:
mask = (outputxlsx['date'] > datetime(year = 2021, month = 10, day = 25))                                                              # Remove dates more than 6 months ago
outputxlsx = outputxlsx.loc[mask]

In [35]:
dfs = DataFrameSummary(outputxlsx)
dfs.columns_stats

,Unnamed: 0,Channel Name,title,date,thumbnailUrl,Views,Description,Duration,Like,Subscribers
counts,23239,23239,23239,23239,23239,23239,23070,23239,23239,23239
uniques,1644,110,20028,23215,23239,16428,14402,4237,5382,108
missing,0,0,0,0,0,0,169,0,0,0
missing_perc,0%,0%,0%,0%,0%,0%,0.73%,0%,0%,0%
types,numeric,categorical,categorical,date,unique,numeric,categorical,categorical,numeric,numeric


In [36]:
outputxlsx.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,23239.0,252.401523,315.330049,0.0,56.0,134.0,287.0,1643.0
Views,23239.0,24499.199578,168791.713659,7.0,3273.5,8300.0,25944.5,24719111.0
Like,23239.0,1526.014028,2771.801317,0.0,207.0,565.0,1616.5,156093.0
Subscribers,23239.0,217070.796420,307517.927542,215.0,41800.0,84600.0,268000.0,1990000.0
